<a href="https://colab.research.google.com/github/tpavan03/Phishing-Detection/blob/main/Phase3CreatingEmbeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas scikit-learn sentence-transformers transformers tensorflow tensorflow-hub gensim torch xgboost openai==0.28.0 tqdm

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [ ]:
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
import xgboost as xgb

# For embeddings
from sentence_transformers import SentenceTransformer
from transformers import BertTokenizer, BertModel, RobertaTokenizer, RobertaModel, AlbertTokenizer, AlbertModel, TFBertForSequenceClassification
import tensorflow_hub as hub
import openai
import gensim
import torch

# For Neural Networks
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, LSTM, Bidirectional, Embedding, Flatten

In [ ]:
df=pd.read_csv("/kaggle/input/pavans/pavan.csv")

In [ ]:

X_total = df.drop(['FILENAME', 'Domain', 'TLD', 'Title', 'label'], axis=1)
y = df['label'].values

In [ ]:
df.shape

(235795, 56)

In [ ]:
X_train_total, X_test_total, y_train, y_test = train_test_split(X_total, y, test_size=0.2, random_state=42, stratify=y)
# Separate 'URL' from other features in training set
X_train = X_train_total['URL'].values
X_train_manual = X_train_total.drop('URL', axis=1).values

# Separate 'URL' from other features in testing set
X_test = X_test_total['URL'].values
X_test_manual = X_test_total.drop('URL', axis=1).values# Separate 'URL' from other features in training set
X_train = X_train_total['URL'].values
X_train_manual = X_train_total.drop('URL', axis=1).values

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

# Fit scaler on training manual features and transform both training and testing manual features
X_train_manual_scaled = scaler.fit_transform(X_train_manual)
X_test_manual_scaled = scaler.transform(X_test_manual)

In [ ]:
# Save the manual features using pickle
with open('X_train_manual_scaled.pkl', 'wb') as f:
  pickle.dump(X_train_manual_scaled,f)

with open('X_test_manual_scaled.pkl', 'wb') as f:
  pickle.dump(X_test_manual_scaled,f)
with open('y_train.pkl', 'wb') as f:
  pickle.dump(y_train,f)

with open('y_test.pkl', 'wb') as f:
  pickle.dump(y_test,f)

In [ ]:
def get_sbert_embeddings(texts):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode(texts, batch_size=32, show_progress_bar=True)
    return embeddings

In [ ]:
def get_bert_cls_embeddings(texts, batch_size=64, max_length=128, model_name='bert-base-uncased', device='cuda'):
    # Convert NumPy array to list if necessary
    if isinstance(texts, np.ndarray):
        texts = texts.tolist()
    elif not isinstance(texts, list):
        texts = list(texts)

    # Ensure all elements are strings, replace non-strings with empty strings
    texts = [str(text) if isinstance(text, str) else "" for text in texts]
    # Initialize tokenizer and model
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertModel.from_pretrained(model_name)
    model.to(device)
    model.eval()

    embeddings = []

    # Disable gradient calculation for inference
    with torch.no_grad():
        for i in tqdm(range(0, len(texts), batch_size), desc="BERT CLS Embeddings"):
            batch_texts = texts[i:i + batch_size]
            # Tokenize the batch
            inputs = tokenizer(batch_texts,
                               padding=True,
                               truncation=True,
                               max_length=max_length,
                               return_tensors='pt')
            # Move inputs to the specified device
            inputs = {key: val.to(device) for key, val in inputs.items()}
            # Forward pass
            outputs = model(**inputs)
            # Extract CLS token embeddings
            cls_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
            embeddings.append(cls_embeddings)

    # Concatenate all embeddings
    embeddings = np.vstack(embeddings)
    return embeddings


In [ ]:
def get_use_embeddings(texts, batch_size=32):
    use_model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
    embeddings = []
    for i in tqdm(range(0, len(texts), batch_size), desc="USE Embeddings", unit="batch"):
        batch_texts = texts[i:i + batch_size]
        batch_embeddings = use_model(batch_texts).numpy()
        embeddings.extend(batch_embeddings)
    return np.array(embeddings)

In [ ]:
import openai
from tqdm import tqdm
import numpy as np

def get_openai_embeddings(texts, api_key, batch_size=100):

    openai.api_key = api_key
    embeddings = []

    # Process texts in batches
    for i in tqdm(range(0, len(texts), batch_size), desc="OpenAI Embeddings"):
        batch_texts = texts[i:i + batch_size]
        try:
            response = openai.Embedding.create(input=batch_texts, model="text-embedding-ada-002")
            batch_embeddings = [item['embedding'] for item in response['data']]
            embeddings.extend(batch_embeddings)
        except Exception as e:
            print(f"Error processing batch {i // batch_size}: {e}")
            # Optionally, handle retries or skip the batch
            continue

    return np.array(embeddings)


In [ ]:
from gensim.utils import simple_preprocess
from tqdm import tqdm
import numpy as np
import gensim.downloader as api

def get_fasttext_embeddings(texts):
    """
    Generates FastText embeddings for a list of texts by averaging word vectors.

    Args:
        texts (List[str]): List of input texts.

    Returns:
        np.ndarray: Array of sentence embeddings.
    """
    # Load FastText KeyedVectors model
    print("Loading FastText model...")
    model = api.load('fasttext-wiki-news-subwords-300')  # 300-dimensional embeddings

    embeddings = []
    vector_size = model.vector_size  # Typically 300

    for text in tqdm(texts, desc="FastText Embeddings", unit="url"):
        # Preprocess the text: tokenize, lowercase, remove punctuation
        tokens = simple_preprocess(text)

        if tokens:
            # Retrieve word vectors for tokens present in the model's vocabulary
            word_vectors = [model.get_vector(token) for token in tokens if token in model]

            if word_vectors:
                # Compute the mean of word vectors to get the sentence embedding
                sentence_vector = np.mean(word_vectors, axis=0)
            else:
                # If no tokens are found in the vocabulary, use a zero vector
                sentence_vector = np.zeros(vector_size)
        else:
            # If text is empty or only contains non-alphanumeric characters, use a zero vector
            sentence_vector = np.zeros(vector_size)

        embeddings.append(sentence_vector)

    return np.array(embeddings)

In [ ]:
def get_roberta_embeddings(texts, batch_size=32, max_length=128, model_name='roberta-base', device='cuda'):
    if isinstance(texts, np.ndarray):
        texts = texts.tolist()
    elif not isinstance(texts, list):
        texts = list(texts)

    # Ensure all elements are strings, replace non-strings with empty strings
    texts = [str(text) if isinstance(text, str) else "" for text in texts]
    # Initialize tokenizer and model
    tokenizer = RobertaTokenizer.from_pretrained(model_name)
    model = RobertaModel.from_pretrained(model_name)
    model.to(device)
    model.eval()

    embeddings = []

    # Disable gradient calculation for inference
    with torch.no_grad():
        for i in tqdm(range(0, len(texts), batch_size), desc="RoBERTa Embeddings"):
            batch_texts = texts[i:i + batch_size]
            # Tokenize the batch
            inputs = tokenizer(batch_texts,
                               padding=True,
                               truncation=True,
                               max_length=max_length,
                               return_tensors='pt')
            # Move inputs to the specified device
            inputs = {key: val.to(device) for key, val in inputs.items()}
            # Forward pass
            outputs = model(**inputs)
            # Extract CLS token embeddings
            cls_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
            embeddings.append(cls_embeddings)

    # Concatenate all embeddings
    embeddings = np.vstack(embeddings)
    return embeddings

In [ ]:
def get_roberta_embeddings(texts, batch_size=32, max_length=128, model_name='roberta-base', device='cuda'):
    if isinstance(texts, np.ndarray):
        texts = texts.tolist()
    elif not isinstance(texts, list):
        texts = list(texts)

    # Ensure all elements are strings, replace non-strings with empty strings
    texts = [str(text) if isinstance(text, str) else "" for text in texts]
    # Initialize tokenizer and model
    tokenizer = RobertaTokenizer.from_pretrained(model_name)
    model = RobertaModel.from_pretrained(model_name)
    model.to(device)
    model.eval()

    embeddings = []

    # Disable gradient calculation for inference
    with torch.no_grad():
        for i in tqdm(range(0, len(texts), batch_size), desc="RoBERTa Embeddings"):
            batch_texts = texts[i:i + batch_size]
            # Tokenize the batch
            inputs = tokenizer(batch_texts,
                               padding=True,
                               truncation=True,
                               max_length=max_length,
                               return_tensors='pt')
            # Move inputs to the specified device
            inputs = {key: val.to(device) for key, val in inputs.items()}
            # Forward pass
            outputs = model(**inputs)
            # Extract CLS token embeddings
            cls_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
            embeddings.append(cls_embeddings)

    # Concatenate all embeddings
    embeddings = np.vstack(embeddings)
    return embeddings

In [ ]:
def get_albert_embeddings(texts, batch_size=32, max_length=128, model_name='albert-base-v2', device='cuda'):
    if isinstance(texts, np.ndarray):
        texts = texts.tolist()
    elif not isinstance(texts, list):
        texts = list(texts)

    # Ensure all elements are strings, replace non-strings with empty strings
    texts = [str(text) if isinstance(text, str) else "" for text in texts]
    # Initialize tokenizer and model
    tokenizer = AlbertTokenizer.from_pretrained(model_name)
    model = AlbertModel.from_pretrained(model_name)
    model.to(device)
    model.eval()

    embeddings = []

    # Disable gradient calculation for inference
    with torch.no_grad():
        for i in tqdm(range(0, len(texts), batch_size), desc="ALBERT Embeddings"):
            batch_texts = texts[i:i + batch_size]
            # Tokenize the batch
            inputs = tokenizer(batch_texts,
                               padding=True,
                               truncation=True,
                               max_length=max_length,
                               return_tensors='pt')
            # Move inputs to the specified device
            inputs = {key: val.to(device) for key, val in inputs.items()}
            # Forward pass
            outputs = model(**inputs)
            # Extract CLS token embeddings
            cls_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
            embeddings.append(cls_embeddings)

    # Concatenate all embeddings
    embeddings = np.vstack(embeddings)
    return embeddings

In [ ]:
embeddings_dict = {}

In [ ]:
print("Generating SBERT embeddings...")
embeddings_dict['SBERT'] = get_sbert_embeddings(X_train)
embeddings_dict['SBERT'].shape

Generating SBERT embeddings...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/5895 [00:00<?, ?it/s]

(188636, 384)

In [ ]:
print("Generating BERT CLS embeddings...")
embeddings_dict['BERT_CLS'] = get_bert_cls_embeddings(X_train)
embeddings_dict['BERT_CLS'].shape

Generating BERT CLS embeddings...


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BERT CLS Embeddings: 100%|██████████| 2948/2948 [08:54<00:00,  5.51it/s]


(188636, 768)

In [ ]:
print("Generating USE embeddings...")
embeddings_dict['USE'] = get_use_embeddings(X_train)
embeddings_dict['USE'].shape

Generating USE embeddings...


USE Embeddings: 100%|██████████| 5895/5895 [00:39<00:00, 147.42batch/s]


(188636, 512)

In [ ]:
print("Generating RoBERTa embeddings...")
embeddings_dict['RoBERTa'] = get_roberta_embeddings(X_train)
embeddings_dict['RoBERTa'].shape

Generating RoBERTa embeddings...


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
RoBERTa Embeddings: 100%|██████████| 5895/5895 [06:36<00:00, 14.85it/s]


(188636, 768)

In [ ]:
print("Generating ALBERT embeddings...")
embeddings_dict['ALBERT'] = get_albert_embeddings(X_train)
embeddings_dict['ALBERT'].shape

Generating ALBERT embeddings...


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

ALBERT Embeddings: 100%|██████████| 5895/5895 [07:47<00:00, 12.62it/s]


(188636, 768)

In [ ]:
test_embeddings_dict = {}

In [ ]:
# SBERT
print("Generating SBERT test embeddings...")
test_embeddings_dict['SBERT'] = get_sbert_embeddings(X_test)
test_embeddings_dict['SBERT'].shape

Generating SBERT test embeddings...


Batches:   0%|          | 0/1474 [00:00<?, ?it/s]

(47159, 384)

In [ ]:
# BERT CLS Token
print("Generating BERT CLS test embeddings...")
test_embeddings_dict['BERT_CLS'] = get_bert_cls_embeddings(X_test)
test_embeddings_dict['BERT_CLS'].shape

Generating BERT CLS test embeddings...


BERT CLS Embeddings: 100%|██████████| 737/737 [02:16<00:00,  5.42it/s]


(47159, 768)

In [ ]:
# USE
print("Generating USE test embeddings...")
test_embeddings_dict['USE'] = get_use_embeddings(X_test)
test_embeddings_dict['USE'].shape

Generating USE test embeddings...


USE Embeddings: 100%|██████████| 1474/1474 [00:10<00:00, 142.71batch/s]


(47159, 512)

In [ ]:
# FastText
print("Generating FastText test embeddings...")
test_embeddings_dict['FastText'] = get_fasttext_embeddings(X_test)
test_embeddings_dict['FastText'].shape

Generating FastText test embeddings...
Loading FastText model...
[=-------------------------------------------------] 3.8% 36.9/958.4MB downloaded

In [ ]:
# RoBERTa
print("Generating RoBERTa test embeddings...")
test_embeddings_dict['RoBERTa'] = get_roberta_embeddings(X_test)
test_embeddings_dict['RoBERTa'].shape

In [ ]:
# ALBERT
print("Generating ALBERT test embeddings...")
test_embeddings_dict['ALBERT'] = get_albert_embeddings(X_test)
test_embeddings_dict['ALBERT'].shape

In [ ]:
# Save the trained model using pickle
with open('embeddings_dict.pkl', 'wb') as f:
  pickle.dump(embeddings_dict, f)
with open('test_embeddings_dict.pkl', 'wb') as f:
  pickle.dump(test_embeddings_dict, f)

In [ ]:
# read the manual features using pickle
with open('X_train_manual_scaled.pkl', 'rb') as f:
  X_train_manual_scaled = pickle.load(f)

with open('X_test_manual_scaled(1).pkl', 'rb') as f:
  X_test_manual_scaled = pickle.load(f)

In [ ]:
with open('embeddings_dict.pkl', 'rb') as f:
  embeddings_dict=pickle.load( f)
with open('test_embeddings_dict.pkl', 'rb') as f:
  test_embeddings_dict=pickle.load(f)

In [ ]:
print(embeddings_dict.keys())

In [ ]:
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'XGBoost': xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    # 'SVM_rbf': SVC(kernel='rbf', probability=True),
     #'SVM_linear': SVC(kernel='linear', probability=True),
    # 'SVM_poly': SVC(kernel='poly', probability=True),
    # 'SVM_sigmoid': SVC(kernel='sigmoid', probability=True),
    #'Decision Tree': DecisionTreeClassifier(),
     #'Random Forest': RandomForestClassifier(n_estimators=100),
    # 'Gradient Boosting': GradientBoostingClassifier(n_estimators=100),
    # 'XGBoost': xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    # 'k-NN': KNeighborsClassifier(n_neighbors=5),
    # 'Feed Forward NN': MLPClassifier(max_iter=1000),
    # Add more neural network models as needed
    # 'Convolutional NN': ...,
    # 'RNN/LSTM': ...,
    # 'Bidirectional LSTM': ...,
    # 'TFBertForSequenceClassification': ...,
}

In [ ]:
# Initialize a list to store results
results_list = []

# Iterate over each embedding
for embedding_name in embeddings_dict.keys():
    print(f"Processing embedding: {embedding_name}")
    X_train_sem_emb = embeddings_dict[embedding_name]
    X_test_sem_emb = test_embeddings_dict[embedding_name]
    # Concatenate training embeddings with manual features
    X_train_emb = np.hstack((X_train_sem_emb, X_train_manual_scaled))

    # Concatenate testing embeddings with manual features
    X_test_emb = np.hstack((X_test_sem_emb, X_test_manual_scaled))
    train_emb_shape = X_train_emb.shape
    test_emb_shape = X_test_emb.shape
    # Iterate over each model
    for model_name, model in models.items():
        print(f"Training model: {model_name} with embedding: {embedding_name}")
        try:
            model.fit(X_train_emb, y_train)
            with open(f'{model_name}embedding{embedding_name}.pkl', 'wb') as f:
              pickle.dump(model, f)
            y_pred = model.predict(X_test_emb)

            # Calculate metrics
            acc = accuracy_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred)
            prec = precision_score(y_test, y_pred)
            rec = recall_score(y_test, y_pred)

            # Append results to the list
            result = {
                'Embedding': embedding_name,
                'Train Emb Shape': train_emb_shape,
                'Test Emb Shape': test_emb_shape,
                'Model': model_name,
                'Accuracy': acc,
                'F1_Score': f1,
                'Precision': prec,
                'Recall': rec
            }
            print(result)
            results_list.append(result)
        except Exception as e:
            print(f"Error training {model_name} with {embedding_name}: {e}")
            results_list.append({
                'Embedding': embedding_name,
                'Model': model_name,
                'Train Emb Shape': train_emb_shape,
                'Test Emb Shape': test_emb_shape,
                'Accuracy': None,
                'F1_Score': None,
                'Precision': None,
                'Recall': None
            })

# Convert results list to DataFrame
results = pd.DataFrame(results_list)


In [ ]:
# Example: Accuracy Matrix
accuracy_matrix = results.pivot(index='Model', columns='Embedding', values='Accuracy')
print("Accuracy Matrix:")
print(accuracy_matrix)

# Similarly, you can create matrices for F1 Score, Precision, Recall
f1_matrix = results.pivot(index='Model', columns='Embedding', values='F1_Score')
precision_matrix = results.pivot(index='Model', columns='Embedding', values='Precision')
recall_matrix = results.pivot(index='Model', columns='Embedding', values='Recall')

# To save the results
results.to_csv('model_embedding_results.csv', index=False)